In [5]:
%pip install --upgrade boto3 botocore

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [6]:
import os
import datetime
import gzip
import shutil
import logging
from pathlib import Path
from dotenv import load_dotenv

# Database libraries
import pymysql
import psycopg2
from pymongo import MongoClient
import sqlite3

# AWS S3
import boto3


In [7]:
# Configure logging
log_file = "db_backup.log"
logging.basicConfig(filename=log_file, 
                    level=logging.INFO, 
                    format='%(asctime)s - %(levelname)s - %(message)s')

def log(msg, level="info"):
    if level == "info":
        logging.info(msg)
    elif level == "error":
        logging.error(msg)
    print(msg)

In [8]:
def test_mysql_connection(host, port, user, password, database):
    try:
        conn = pymysql.connect(host=host, port=port, user=user, password=password, database=database)
        conn.close()
        log(f"MySQL connection to '{database}' successful.")
        return True
    except Exception as e:
        log(f"MySQL connection error: {e}", "error")
        return False

def test_postgresql_connection(host, port, user, password, database):
    try:
        conn = psycopg2.connect(host=host, port=port, user=user, password=password, dbname=database)
        conn.close()
        log(f"PostgreSQL connection to '{database}' successful.")
        return True
    except Exception as e:
        log(f"PostgreSQL connection error: {e}", "error")
        return False

def test_mongodb_connection(uri):
    try:
        client = MongoClient(uri)
        client.admin.command('ping')
        log("MongoDB connection successful.")
        return True
    except Exception as e:
        log(f"MongoDB connection error: {e}", "error")
        return False

def test_sqlite_connection(db_path):
    try:
        conn = sqlite3.connect(db_path)
        conn.close()
        log(f"SQLite connection to '{db_path}' successful.")
        return True
    except Exception as e:
        log(f"SQLite connection error: {e}", "error")
        return False

In [9]:
def backup_mysql(host, port, user, password, database, backup_path):
    timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
    backup_file = f"{backup_path}/{database}_{timestamp}.sql"
    os.system(f"mysqldump -h {host} -P {port} -u {user} -p{password} {database} > {backup_file}")
    compress_file(backup_file)
    log(f"MySQL backup completed: {backup_file}.gz")
    return backup_file + ".gz"

def backup_postgresql(host, port, user, password, database, backup_path):
    timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
    backup_file = f"{backup_path}/{database}_{timestamp}.sql"
    os.system(f"PGPASSWORD={password} pg_dump -h {host} -p {port} -U {user} -d {database} -F c -b -v -f {backup_file}")
    compress_file(backup_file)
    log(f"PostgreSQL backup completed: {backup_file}.gz")
    return backup_file + ".gz"

def backup_sqlite(db_path, backup_path):
    timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
    db_name = os.path.basename(db_path).split(".")[0]
    backup_file = f"{backup_path}/{db_name}_{timestamp}.db"
    shutil.copy2(db_path, backup_file)
    compress_file(backup_file)
    log(f"SQLite backup completed: {backup_file}.gz")
    return backup_file + ".gz"

def backup_mongodb(uri, db_name, backup_path):
    timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
    backup_file = f"{backup_path}/{db_name}_{timestamp}"
    os.system(f"mongodump --uri='{uri}' --db {db_name} --out {backup_file}")
    shutil.make_archive(backup_file, 'gztar', backup_file)
    shutil.rmtree(backup_file)  # remove uncompressed dump
    log(f"MongoDB backup completed: {backup_file}.tar.gz")
    return backup_file + ".tar.gz"

def compress_file(file_path):
    with open(file_path, 'rb') as f_in, gzip.open(file_path + '.gz', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)
    os.remove(file_path)  # remove original

In [10]:
def upload_to_s3(file_path, bucket_name, s3_key):
    s3 = boto3.client('s3')
    try:
        s3.upload_file(file_path, bucket_name, s3_key)
        log(f"Backup uploaded to S3: s3://{bucket_name}/{s3_key}")
        return True
    except Exception as e:
        log(f"S3 upload failed: {e}", "error")
        return False

In [11]:
def restore_mysql(host, port, user, password, database, backup_file_gz):
    backup_file = backup_file_gz.replace('.gz', '')
    with gzip.open(backup_file_gz, 'rb') as f_in, open(backup_file, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)
    os.system(f"mysql -h {host} -P {port} -u {user} -p{password} {database} < {backup_file}")
    os.remove(backup_file)
    log(f"MySQL restore completed from: {backup_file_gz}")

In [12]:
backup_dir = "backups"
os.makedirs(backup_dir, exist_ok=True)

# MySQL backup example
backup_mysql(host="localhost", port=3306, user="root", password="rootpass", database="test_db", backup_path=backup_dir)

# PostgreSQL backup example
# backup_postgresql(host="localhost", port=5432, user="postgres", password="pass", database="test_db", backup_path=backup_dir)

# SQLite backup example
# backup_sqlite(db_path="example.db", backup_path=backup_dir)

# MongoDB backup example
# backup_mongodb(uri="mongodb://localhost:27017", db_name="test_db", backup_path=backup_dir)

MySQL backup completed: backups/test_db_20250904191852.sql.gz


'backups/test_db_20250904191852.sql.gz'